<a href="https://colab.research.google.com/github/MadKauSing/GuessFlat/blob/main/GuessFlat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MADKAUSING GUESSFLAT
```
Ayush Singh          -PES2UG20C0S81
Ayushmaan Kaushik    -PES2UG20CS082
Bhavini Madhuranath  -PES2UG20CS088
```

### An indian music genre classifier
